# Riddler Classic 2017-04-22

Let's start with a simpler version of the problem. If we draw two cards instead of ten, what is the best strategy? Clearly, the best strategy is to choose the option, end or continue, that maximizes the chance of winning.

Let $c_i$ be the number on $i+1$th to last card. In the case above, the probablity of winning if ending the game after the first card is $\frac{c_1-1}{99}$. If you continue, the probability of winning is $\frac{100-c_1}{99}$.

More generally, if you are drawing $k$ cards from a deck of $n$ cards numbered 1 through $n$ and you get a new maximum $c_1$ when drawing the $(k-1)$th card, the probability of winning if you stop is $\frac{c_1-(k-1)}{n-(k-1)}$ and if you continue $\frac{n-c_1}{n-(k-1)}$.

What about for $i > 1$? The same basic strategy applies here, we choose the option that maximizes our chances of winning. Notice that we only need to make a decision if the current card is the highest so far, otherwise we will have to continue. If we stop, the probability of winning is easy to calculate. We need to calculate the probability that at none of the remaining cards is greater than $c_i$. This is probably easiest to write as a recursive formula.

In [1]:
def prob_win_if_stop(num_cards, cards_to_take, cards_remaining, curr_card_val):
    if cards_remaining == 0:
        return 1.0
    cards_taken = cards_to_take - cards_remaining
    return (curr_card_val - cards_taken) / (num_cards - cards_taken) * prob_win_if_stop(
                    num_cards, cards_to_take, cards_remaining-1, curr_card_val)

What about continuing? This is more challenging, because we need to account for all the possible cases. Will we stop or continue after the next card? The answer depends on what that card will be and hence we need to take that into account. If the next card is lower than the current, we have no choice, but if it's higher we need to make a decision. Once again it's probably best to use a recursive formula.

In [2]:
def prob_win_if_continue(num_cards, cards_to_take, cards_remaining, curr_max):
    if cards_remaining == 1:
        return 1 - prob_win_if_stop(num_cards, cards_to_take, cards_remaining, curr_max)
    cards_taken = cards_to_take - cards_remaining
    cards_below_max = curr_max - cards_taken
    
    prob_win_if_below_max = prob_win_if_continue(
        num_cards, cards_to_take, cards_remaining-1, curr_max)
    
    probs_win_if_above_max = [max(
            prob_win_if_continue(num_cards, cards_to_take, cards_remaining-1, i),
            prob_win_if_stop(num_cards, cards_to_take, cards_remaining-1, i)
        ) for i in range(curr_max+1, num_cards+1)]
    
    return (cards_below_max*prob_win_if_below_max + sum(probs_win_if_above_max)
        ) / (num_cards - cards_taken)

We can now formulate our strategy like this:

> Stop if and only if the current card is the highest so far _and_ its value is greater than or equal to a threshold value.

Below is a simple function to compute the threshold.

In [3]:
def stop_threshold(num_cards, cards_to_take, cards_remaining):
    for i in range(100, 0, -1):
        if (
            prob_win_if_continue(num_cards, cards_to_take, cards_remaining, i)
            > prob_win_if_stop(num_cards, cards_to_take, cards_remaining, i)):
                return i + 1

For the given problem, the thresholds for card 1 through 9 are

In [4]:
[stop_threshold(100, 10, i) for i in range(9, 0, -1)]

[93, 92, 91, 89, 87, 84, 80, 72, 55]